<a href="https://colab.research.google.com/github/sdhar2020/Visualization-Exploratory-Analysis/blob/master/NYC_School_SAT_Step_1_Data_Cleaning_%26_Merging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
One of the most controversial issues in the U.S. educational system is the efficacy of standardized tests, and whether they're unfair to certain groups. Given our prior knowledge of this topic, investigating the correlations between SAT scores and demographics might be an interesting angle to take. We could correlate SAT scores with factors like race, gender, income, and more.

The SAT, or Scholastic Aptitude Test, is an exam that U.S. high school students take before applying to college. Colleges take the test scores into account when deciding who to admit, so it's fairly important to perform well on it.

In order to investigate the key drivers of performance we shall use the following publicly available datasets and combine them to analyze performance based on attributes:



* SAT scores by school - SAT scores for each high school in New York City
* Class size - Information on class size for each school
* AP test results - Advanced Placement (AP) exam results for each high school (passing an optional AP exam in a particular subject can earn a student college credit in that subject)
* Graduation outcomes - The percentage of students who graduated, and other outcome information
* Demographics - Demographic information for each school
School survey - Surveys of parents, teachers, and students at each school
* High School Directory; school details



## Installing sodapy

The NYC school data is available at https://opendata.cityofnewyork.us/data/ and available via the Socrata Open Data API. Sodapy package is the package installed run time to download the data

In [ ]:

!pip install sodapy

## Loading API data in Data frames

Manually browsed the data set keys on the web site and built a dictionary with the data frame name and the keys.
Loop through, read the data, cnvert into dataframe and stored as dictionary key, value pair in pandas

In [ ]:
import pandas as pd
from sodapy import Socrata

data = {}
datakey = {'ap_2010':'itfs-ms3e',
          'class_size':'urz7-pzb3',
          'demographics':'ihfw-zy9j',
          'graduation':'vh2h-md7a',
          'hs_directory':'n3p6-zve2',
          'sat_results':'f9bf-2cp4',
          'd75_survey': 'kfrx-hgf8'}

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
#client = Socrata("data.cityofnewyork.us", None)
client = Socrata('data.cityofnewyork.us',
                 'huqpRxRLzufV4wV0kQAmqEt1D',
                 username="sdhar.colab@gmail.com",
                 password="Aug3!2019")
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
for k,v in datakey.items():
    results = client.get(v, limit=200000)
    # Convert to pandas DataFrame
    df = pd.DataFrame.from_records(results)
    print('name:',k, 'shape:', df.shape)
    data[k]= df

name: ap_2010 shape: (258, 5)
name: class_size shape: (27611, 16)
name: demographics shape: (10075, 38)
name: graduation shape: (25096, 23)
name: hs_directory shape: (435, 69)
name: sat_results shape: (478, 6)
name: d75_survey shape: (56, 105)


## Investigating data


In [ ]:
for key in data:
    df= data[key]
    print('__________________________________________')
    print(key,':', '\n')
    print(df.shape)
    print(df.head(2).transpose())
    print('__________________________________________')

__________________________________________
ap_2010 : 

(258, 5)
                                                                 0                       1
dbn                                                         01M448                  01M450
schoolname                            UNIVERSITY NEIGHBORHOOD H.S.  EAST SIDE COMMUNITY HS
ap_test_takers_                                                 39                      19
total_exams_taken                                               49                      21
number_of_exams_with_scores_3_4_or_5                            10                     NaN
__________________________________________
__________________________________________
class_size : 

(27611, 16)
                                                             0                          1
csd                                                          1                          1
borough                                                      M                          M
school_

## 1 More File

There are 2 more files that are csv that needs to be included. School survey information from 2011...

In [ ]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/Data/NYC School Data/2011_School_Survey.zip", 'r')
zip_ref.extractall("/content/drive/My Drive/Data/NYC School Data/")
zip_ref.close()

folder = "/content/drive/My Drive/Data/NYC School Data/2011 data files online/"
filen = folder+'masterfile11_gened_final.txt'
all_survey = pd.read_csv(filen, delimiter= '\t', encoding = 'windows-1252')
df = data['d75_survey']
survey = pd.concat([all_survey, df], axis=0, sort = True)
print(survey.shape)

data['survey']= survey

# Final Data Frame Dictionary

# for key in data:
#     df= data[key]
#     print('__________________________________________')
#     print(key,':', '\n')
#     print(df.shape)
#     print(df.head(1).transpose())
#     print('__________________________________________')
del data['d75_survey']

for key in data:
  print(key)

(1702, 2014)
ap_2010
class_size
demographics
graduation
hs_directory
sat_results
survey


The data source provides us a data dictionary [here](https://data.cityofnewyork.us/Education/2011-NYC-School-Survey/mnz3-dyi8) Based on the description a subset is deemed important

In [ ]:
req_cols= ["dbn", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", 
           "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "eng_t_11",
           "aca_t_11", "saf_s_11", "com_s_11", 
           "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]

print(survey[req_cols].head(2).transpose())

                 0       1
dbn         01M015  01M019
rr_s           NaN     NaN
rr_t            88     100
rr_p            60      60
N_s            NaN     NaN
N_t             22      34
N_p             90     161
saf_p_11       8.5     8.4
com_p_11       7.6     7.6
eng_p_11       7.5     7.6
aca_p_11       7.8     7.8
saf_t_11       7.5     8.6
com_t_11       7.8     8.5
eng_t_11       7.6     8.9
aca_t_11       7.9     9.1
saf_s_11       NaN     NaN
com_s_11       NaN     NaN
eng_s_11       NaN     NaN
aca_s_11       NaN     NaN
saf_tot_11       8     8.5
com_tot_11     7.7     8.1
eng_tot_11     7.5     8.2
aca_tot_11     7.9     8.4


## Case of missing dbn number
When we explored all of the data sets, we noticed that some of them, like class_size and hs_directory, don't have a DBN column. hs_directory does have a dbn column, though, so we can just rename it.

From looking at these rows, we can tell that the DBN in the sat_results data is just a combination of the CSD and SCHOOL CODE columns in the class_size data. The main difference is that the DBN is padded, so that the CSD portion of it always consists of two digits. That means we'll need to add a leading 0 to the CSD if the CSD is less than two digits long.



In [ ]:
data["hs_directory"]["DBN"] = data["hs_directory"]["dbn"]

def pad_csd(num):
    return str(num).zfill(2)
    
data["class_size"]["padded_csd"] = data["class_size"]["csd"].apply(pad_csd)
data["class_size"]["dbn"] = data["class_size"]["padded_csd"] + data["class_size"]["school_code"]
print(data["class_size"].head())

  csd borough school_code  ... schoolwide_pupil_teacher_ratio padded_csd     dbn
0   1       M        M015  ...                            NaN         01  01M015
1   1       M        M015  ...                            NaN         01  01M015
2   1       M        M015  ...                            NaN         01  01M015
3   1       M        M015  ...                            NaN         01  01M015
4   1       M        M015  ...                            NaN         01  01M015

[5 rows x 18 columns]


## Initial manipulation

Using DBN as the primary key we should be able to put all the data frames together. Before we do, let's take some time to calculate variables that will be useful in our analysis. We've already discussed one such variable -- a column that totals up the SAT scores for the different sections of the exam. This will make it much easier to correlate scores with demographic factors because we'll be working with a single number, rather than three different ones.





In [ ]:
data['sat_results'].columns

Index(['dbn', 'school_name', 'num_of_sat_test_takers',
       'sat_critical_reading_avg_score', 'sat_math_avg_score',
       'sat_writing_avg_score'],
      dtype='object')

In [ ]:
sat_scr = ['sat_critical_reading_avg_score','sat_math_avg_score', 'sat_writing_avg_score']

for c in sat_scr:
    data['sat_results'][c] = pd.to_numeric(data["sat_results"][c], errors="coerce")

data['sat_results']['sat_score']=0.0
for c in sat_scr:
    data['sat_results']['sat_score'] += data['sat_results'][c]

Next, we'll want to parse the latitude and longitude coordinates for each school. This will enable us to map the schools and uncover any geographic patterns in the data. The coordinates are currently in the text field Location 1 in the hs_directory data set.

The column is a python dictionary that has home address and lat long stored as various key-value pairs 

In [ ]:
v=data['hs_directory']['location_1'][0]
print(v['latitude'])


40.589238098


In [ ]:

data['hs_directory']['lat'] = data['hs_directory']['location_1'].apply(lambda x: x.get('latitude'))
data['hs_directory']['long'] = data['hs_directory']['location_1'].apply(lambda x: x.get('longitude'))
data["hs_directory"]["lat"] = pd.to_numeric(data["hs_directory"]["lat"], errors="coerce")
data["hs_directory"]["long"] = pd.to_numeric(data["hs_directory"]["long"], errors="coerce")

## Cleaning Each Data File


We are trying to investigate factors impacting SAT scores and related demographic variables

The first data set that we'll condense is class_size. school has multiple values for GRADE, PROGRAM TYPE, CORE SUBJECT (MS CORE and 9-12 ONLY), and CORE COURSE (MS CORE and 9-12 ONLY)



```
# This is formatted as code
```





In [ ]:
class_size = data['class_size']
class_size.columns

Index(['csd', 'borough', 'school_code', 'school_name', 'grade_',
       'program_type', 'core_subject_ms_core_and_9_12_only_',
       'core_course_ms_core_and_9_12_only_', 'service_category_k_9_only_',
       'number_of_students_seats_filled', 'number_of_sections',
       'average_class_size', 'size_of_smallest_class', 'size_of_largest_class',
       'data_source', 'schoolwide_pupil_teacher_ratio', 'padded_csd', 'dbn'],
      dtype='object')

In [ ]:
class_size['grade_'].value_counts()
# We only select '09-12' as we are looking at a High School education
grd_flg= class_size['grade_'] == '09-12'
class_size = class_size[grd_flg]

In [ ]:
class_size['program_type'].value_counts()
# We select only programs type of 'Gen Ed'
prg_flg = class_size['program_type']== 'GEN ED'
class_size = class_size[prg_flg]

'core_course_ms_core_and_9_12_only_' and 'core_subject_ms_core_and_9_12_only_' lead to nonunique dbns.

We want our class size data to include every single class a school offers -- not just a subset of them. What we can do is take the average across all of the classes a school offers.
We need to aggregate the data at a dbn level



In [ ]:
import numpy as np
num_fields= ['number_of_sections', 'number_of_students_seats_filled',
             'average_class_size', 'schoolwide_pupil_teacher_ratio', 
             'size_of_largest_class', 'size_of_smallest_class']
class_size[num_fields] = class_size[num_fields].apply(lambda x: pd.to_numeric(x, errors = 'coerce'))
class_size = class_size.groupby('dbn').agg(np.mean)
class_size.reset_index(inplace= True)
data['class_size'] = class_size
print(data['class_size'].head())

      dbn  ...  schoolwide_pupil_teacher_ratio
0  01M292  ...                             NaN
1  01M332  ...                             NaN
2  01M378  ...                             NaN
3  01M448  ...                             NaN
4  01M450  ...                             NaN

[5 rows x 7 columns]


We will now look at demographics. Our objectve would be extract only the information for school year is 20112012


In [ ]:
demographics = data['demographics']
print(demographics.shape)
demographics.columns
demographics['schoolyear'].value_counts()
demographics = demographics[demographics['schoolyear']=='20112012']
print(demographics.shape)
data['demographics']= demographics

(10075, 38)
(1509, 38)


Finally, we'll need to condense the graduation data set. The Demographic and Cohort columns are what prevent dbn from being unique in the graduation data. A Cohort appears to refer to the year the data represents, and the Demographic appears to refer to a specific demographic group. In this case, we want to pick data from the most recent Cohort available, which is 2006. We also want data from the full cohort, so we'll only pick rows where Demographic is Total Cohort

In [ ]:
data["graduation"] = data["graduation"][data["graduation"]["cohort"] == '2006']
data["graduation"] = data["graduation"][data["graduation"]["demographic"] == 'Total Cohort']
#print(data["graduation"].head())
num_vars= [x for x in data["graduation"].columns.tolist() if x.endswith('_n')]
graduation = data['graduation']
graduation[num_vars] = graduation[num_vars].apply(lambda x: pd.to_numeric(x, errors = 'coerce'))
data['graduation']= graduation

Convert the Advanced Placement (AP) test scores from strings to numeric values.

In [ ]:
ap_2010 = data['ap_2010']
ap_2010.columns
num_vars= ['ap_test_takers_', 'number_of_exams_with_scores_3_4_or_5', 'total_exams_taken']
ap_2010[num_vars] = ap_2010[num_vars].apply(lambda x: pd.to_numeric(x, errors = 'coerce'))
data['ap_2010']= ap_2010

## Merging Data Files
We'll be using the dbn column to identify matching rows across data sets. There may be dbn values that exist in one data set but not in another. This is partly because the data is from different years. Each data set also has inconsistencies in terms of how it was gathered.

We'll merge two data sets at a time. For example, we'll merge sat_results and hs_directory, then merge the result with ap_2010, then merge the result of that with class_size. We'll continue combining data sets in this way until we've merged all of them. Afterwards, we'll have roughly the same number of rows, but each row will have columns from all of the data sets.

In [ ]:
combined = data["sat_results"]
print(combined.shape)
combined = pd.merge(combined,data['ap_2010'], how= 'left', on= 'dbn')
combined = pd.merge(combined,data['graduation'], how= 'left', on= 'dbn')
print(combined.shape)

(478, 7)
(479, 33)


In [ ]:
keys = ['class_size', 'demographics', 'survey', 'hs_directory']

for k in keys:
    combined = pd.merge(combined,data[k], how= 'inner', on= 'dbn')

print(combined.shape)

(363, 2160)


The number of records has fallen from 478 to 363. This is because pandas couldn't find the dbn values that existed in sat_results in the other data sets. While this is worth investigating, we're currently looking for high-level correlations, so we don't need to dive into which dbns are missing

Fill in any missing values in combined with the means of the respective columns

In [ ]:
col_mean= combined.mean()
cols= col_mean.index.values.tolist()

combined = combined.fillna(combined.mean())
combined = combined.fillna(0)

print(combined.head(5))

      dbn  ...       long
0  01M292  ... -73.985260
1  01M448  ... -73.984797
2  01M450  ... -73.983041
3  01M509  ... -73.985673
4  01M539  ... -73.979426

[5 rows x 2160 columns]


School Distrct extraction

In [ ]:
combined['school_dist'] = combined['DBN'].apply(lambda x: x[0:2])
combined['school_dist'].head()

0    01
1    01
2    01
3    01
4    01
Name: school_dist, dtype: object

## Visual insights into the SAT performance
Now that we have all the information in one place we want to visuaize the data and draw some correlations etc



In [ ]:
print(combined.shape)
print(combined.dtypes[combined.dtypes== 'float64'].index.to_list())

(363, 2161)
['sat_critical_reading_avg_score', 'sat_math_avg_score', 'sat_writing_avg_score', 'sat_score', 'ap_test_takers_', 'total_exams_taken', 'number_of_exams_with_scores_3_4_or_5', 'total_grads_n', 'total_regents_n', 'advanced_regents_n', 'regents_w_o_advanced_n', 'local_n', 'still_enrolled_n', 'dropped_out_n', 'number_of_students_seats_filled', 'number_of_sections', 'average_class_size', 'size_of_smallest_class', 'size_of_largest_class', 'schoolwide_pupil_teacher_ratio', 'fl_percent', 'N_p', 'N_s', 'N_t', 'aca_p_10', 'aca_p_11', 'aca_s_10', 'aca_s_11', 'aca_t_10', 'aca_t_11', 'aca_tot_10', 'aca_tot_11', 'borough', 'com_p_10', 'com_p_11', 'com_s_10', 'com_s_11', 'com_t_10', 'com_t_11', 'com_tot_10', 'com_tot_11', 'd75', 'eng_p_10', 'eng_p_11', 'eng_s_10', 'eng_s_11', 'eng_t_10', 'eng_t_11', 'eng_tot_10', 'eng_tot_11', 'enrollment', 'highschool', 'location', 'n_p', 'n_s', 'n_t', 'p_N_q10a', 'p_N_q10b', 'p_N_q10c', 'p_N_q10d', 'p_N_q10e', 'p_N_q10f', 'p_N_q10g', 'p_N_q10h', 'p_N_q1

In [ ]:
fileout = folder+'combined.pkl'
pd.to_pickle(combined, fileout)